In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [44.0 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,204 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/mult

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('Homes')


In [6]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("SELECT YEAR(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price FROM Homes WHERE bedrooms == 4 GROUP BY YEAR(date_built) ORDER BY year_built").show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|296800.75|
|      2011| 302141.9|
|      2012|298233.42|
|      2013|299999.39|
|      2014|299073.89|
|      2015|307908.86|
|      2016|296050.24|
|      2017|296576.69|
+----------+---------+



In [8]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql("""SELECT YEAR(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
      FROM Homes
      WHERE bedrooms == 3 AND bathrooms == 3
      GROUP BY YEAR(date_built) ORDER BY year_built""").show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [10]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql("""SELECT YEAR(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
      FROM Homes
      WHERE bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND sqft_living >= 2000
      GROUP BY YEAR(date_built) ORDER BY year_built""").show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [13]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

# spark.sql("""SELECT YEAR(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price, view
#       FROM Homes
#       WHERE bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND price >= 350000
#       GROUP BY YEAR(date_built), view
#       ORDER BY year_built, view""").show()


spark.sql("""SELECT ROUND(AVG(price), 2) AS avg_price, view
      FROM Homes
      WHERE price >= 350000
      GROUP BY view
      ORDER BY view""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+---------+----+
|avg_price|view|
+---------+----+
|403848.51|   0|
|401044.25|   1|
|401868.43|  10|
|1026669.5| 100|
|399548.12|  11|
|401501.32|  12|
|398917.98|  13|
|398570.03|  14|
| 404673.3|  15|
|399586.53|  16|
|398474.49|  17|
|399332.91|  18|
|398953.17|  19|
|397389.25|   2|
|399522.81|  20|
|399758.88|  21|
|402022.68|  22|
|403411.92|  23|
|400284.92|  24|
|401298.69|  25|
+---------+----+
only showing top 20 rows

--- 1.076472520828247 seconds ---


In [18]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE Homes")


DataFrame[]

In [19]:
# 8. Check if the table is cached.
spark.catalog.isCached('Homes')

True

In [20]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""SELECT ROUND(AVG(price), 2) AS avg_price, view
      FROM Homes
      WHERE price >= 350000
      GROUP BY view
      ORDER BY view""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+---------+----+
|avg_price|view|
+---------+----+
|403848.51|   0|
|401044.25|   1|
|401868.43|  10|
|1026669.5| 100|
|399548.12|  11|
|401501.32|  12|
|398917.98|  13|
|398570.03|  14|
| 404673.3|  15|
|399586.53|  16|
|398474.49|  17|
|399332.91|  18|
|398953.17|  19|
|397389.25|   2|
|399522.81|  20|
|399758.88|  21|
|402022.68|  22|
|403411.92|  23|
|400284.92|  24|
|401298.69|  25|
+---------+----+
only showing top 20 rows

--- 0.7047739028930664 seconds ---


In [21]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("/content/drive/My Drive/Module 22 HW")

In [23]:
# 11. Read the parquet formatted data.
df_read = spark.read.parquet("/content/drive/My Drive/Module 22 HW")

In [24]:
# 12. Create a temporary table for the parquet data.
df_read.createOrReplaceTempView('home_sales')

In [25]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""SELECT ROUND(AVG(price), 2) AS avg_price, view
      FROM home_sales
      WHERE price >= 350000
      GROUP BY view
      ORDER BY view""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+---------+----+
|avg_price|view|
+---------+----+
|403848.51|   0|
|401044.25|   1|
|401868.43|  10|
|1026669.5| 100|
|399548.12|  11|
|401501.32|  12|
|398917.98|  13|
|398570.03|  14|
| 404673.3|  15|
|399586.53|  16|
|398474.49|  17|
|399332.91|  18|
|398953.17|  19|
|397389.25|   2|
|399522.81|  20|
|399758.88|  21|
|402022.68|  22|
|403411.92|  23|
|400284.92|  24|
|401298.69|  25|
+---------+----+
only showing top 20 rows

--- 1.4287915229797363 seconds ---


In [26]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")



DataFrame[]

In [27]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False